# Human Activity Recognition Using WiFi Signals

## Overview
Human Activity Recognition (HAR) using WiFi signals leverages the unique properties of wireless channel variations to detect different activities.

## Data Format
- **WiFi signal data** is similar to image data in structure, represented in the shape `(channels, height, width)`, but with a different interpretation:
  - `channels` → **channel**
  - `height` → **Time Steps**
  - `width` → **Antenna Pairs (transmitter-receiver combinations)**
- **Labels** represent a predefined set of classes, as is typical in classification tasks.

# Reading Data

In [1]:
#1
import kagglehub
path = kagglehub.dataset_download("alihabibullah/question-2-data")

print("Path to dataset files:", path)

100%|██████████| 204M/204M [00:04<00:00, 44.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/alihabibullah/question-2-data/versions/1


In [2]:
import os
os.listdir(path)

['WiFiSensingDataset.pt']

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, TensorDataset
from torch.optim import Adam

# Load the .pt file
data = torch.load(f"{path}/WiFiSensingDataset.pt")

<ipython-input-3-0656954f04d1>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(f"{path}/WiFiSensingDataset.pt")


In [4]:

# Convert labels to LongTensor
data['y_train'] = data['y_train'].type(torch.LongTensor)
data['y_test'] = data['y_test'].type(torch.LongTensor)

# Concatenate train and test data
inputs = torch.cat((data['X_train'], data['X_test']))
targets = torch.cat((data['y_train'], data['y_test']))

# Create TensorDataset
mydataset = TensorDataset(inputs, targets)

# Split into train and test datasets
train_size = int(0.8 * len(mydataset))
test_size = len(mydataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(mydataset, [train_size, test_size])

# DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Task 1: Analyze the Dataset ( Stored in `data`)

1. **Determine the number of unique labels** in the dataset.  

2. **Determine the shape of the input data** (number of samples and features).  

3. **Find the maximum value** in the dataset.  

4. **Find the minimum value** in the dataset.  

In [5]:
# -Determine the shape of the input data (number of samples and features)
shape_input = inputs[0].shape
print(f"Shape inputs: {shape_input}")

# -Determine the number of unique labels
unique_labels = torch.unique(targets)
num_unique_labels = len(unique_labels)
print(f"Num unique labels: {num_unique_labels}")

# -Find the maximum value in the dataset
max_value = torch.max(inputs).item()
max_target_value = torch.max(targets).item()
print(f"Maximum inputs: {max_value}")
print(f"Maximum targets: {max_target_value}")

# -Find the minimum value in the dataset
min_value = torch.min(inputs).item()
min_target_value = torch.min(targets).item()
print(f"Minimum inputs: {min_value}")
print(f"Minimum targets: {min_target_value}")

Shape inputs: torch.Size([1, 250, 90])
Num unique labels: 7
Maximum inputs: 1.0
Maximum targets: 6
Minimum inputs: 0.0
Minimum targets: 0


# Task 2: Build and Evaluate a Neural Network

1. **Design a Neural Network (Maximum 5 Layers)**  
   Build a compact neural network with no more than 5 layers. Clearly specify the type of each layer (e.g., Dense, Conv2D) and any activation functions used.

2. **Evaluate Your Model**  
   Train your network on the provided dataset and report the evaluation metrics (e.g., accuracy, loss). Discuss the performance of your model and any challenges faced during training.


In [6]:
import torch.nn as nn
import torch.optim as optim

class NN4Layer(nn.Module):
  def __init__(self, num_inp):

    super(NN4Layer, self).__init__()

    self.layer_1 = nn.Linear(num_inp, 512)
    self.layer_2 = nn.Linear(512, 256)
    self.layer_3 = nn.Linear(256, 128)
    self.layer_4 = nn.Linear(128, 10)

    self.hidden_activation = nn.ReLU()

  def forward(self, x):

    z1 = self.layer_1(x)
    a1 = self.hidden_activation(z1)

    z2 = self.layer_2(a1)
    a2 = self.hidden_activation(z2)

    z3 = self.layer_3(a2)
    a3 = self.hidden_activation(z3)

    z4 = self.layer_4(a3)

    return z4

In [7]:
# Training configuration
num_epochs = 20
lr = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'


train_losses = []
val_losses = []

# Initialize model
model = NN4Layer(1*250*90)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()  # multi-class

model.to(device)

print(f'Using device {device}')

Using device cpu


In [8]:
model.train()
for epoch in range(num_epochs):
  total_loss = 0
  for features,labels in train_loader:
    optimizer.zero_grad()
    # ReShape
    features = features.reshape(-1,250*90).to(device)
    labels = labels.to(device)
    # forward: prediction
    prediction = model(features)
    loss = criterion(prediction, labels)
    total_loss += loss
  # backward
    loss.backward()
  # update weights and take a step
    optimizer.step()


  print(f'Epoch {epoch+ 1} and the loss {total_loss/len(train_loader)}')


Epoch 1 and the loss 1.9666107892990112
Epoch 2 and the loss 1.783217191696167
Epoch 3 and the loss 1.6803579330444336
Epoch 4 and the loss 1.5769826173782349
Epoch 5 and the loss 1.5370591878890991
Epoch 6 and the loss 1.4663300514221191
Epoch 7 and the loss 1.395700454711914
Epoch 8 and the loss 1.324356198310852
Epoch 9 and the loss 1.3826854228973389
Epoch 10 and the loss 1.2359778881072998
Epoch 11 and the loss 1.2868398427963257
Epoch 12 and the loss 1.191057562828064
Epoch 13 and the loss 1.1548160314559937
Epoch 14 and the loss 1.1275105476379395
Epoch 15 and the loss 1.0915288925170898
Epoch 16 and the loss 1.0967615842819214
Epoch 17 and the loss 1.0997689962387085
Epoch 18 and the loss 1.044012427330017
Epoch 19 and the loss 0.9838751554489136
Epoch 20 and the loss 0.9661014676094055


to test

In [9]:
from sklearn.metrics import accuracy_score
model.eval()  # Set the model to evaluation mode
y_true, y_pred = [], []

# Testing loop
with torch.no_grad():
    for features, labels in test_loader:
        features = features.reshape(-1, 250*90).to(device)
        labels = labels.to(device)

        # Forward pass
        prediction = model(features)

        # Get the predicted labels (class with max probability)
        _, predicted_labels = torch.max(prediction, 1)

        # Collect true and predicted labels
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted_labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy on test set: {accuracy * 100:.2f}%')


Accuracy on test set: 67.33%


thanks to Ahmed Y. Radwan
